In [83]:
import numpy as np
import pandas as pd
from astropy.table import Table
from astropy.io.fits import getdata
from astropy.time import Time
from astropy.io import fits

In [84]:
df = pd.read_csv('csv/straycats2_merged.csv')
df['SEQID'] = pd.to_numeric(df['SEQID'])

df['Module'] = [mod.strip() for mod in df['Module']]
df.head()   

,SL Target,SEQID,Module,Primary Target,Exposure (s),RA,DEC,PA,Unnamed: 8,In old sheet?,Notes
0,GX 5-1,90701331002,A,GRO_J1750m27,29886.3108,267.2525,-26.6752,335.8924,NaN,NaN,NaN
1,GR,90701331002,A,GRO_J1750m27,29886.3108,267.2525,-26.6752,335.8924,NaN,NaN,NaN
2,Crab,10702305002,A,Crab_sl_PA330,23410.3679,81.8356,21.1272,330.7766,NaN,NaN,NaN
3,GX 17+2,90701306002,A,Swift_182251m135218,45107.2367,275.7079,-13.8830,157.8715,NaN,NaN,NaN
4,1A 0535+262,90601036001,A,1A_0535p262_SLPA140,2036.1913,86.2426,27.5165,138.9395,NaN,NaN,NaN


In [85]:
greenlist = ['SL Target', 'SEQID', 'Module', 'Primary Target', 'Exposure (s)', 'RA',
       'DEC', 'Notes']
for col in df.columns:
    if col not in greenlist:
        df = df.drop(axis=1, labels=col)
        
# Drop everything with NaN in the SL Target column
df = df.dropna(subset=['SL Target'])
df = df.rename(columns={"Exposure (s)": "Exposure"})

df.head()

,SL Target,SEQID,Module,Primary Target,Exposure,RA,DEC,Notes
0,GX 5-1,90701331002,A,GRO_J1750m27,29886.3108,267.2525,-26.6752,NaN
1,GR,90701331002,A,GRO_J1750m27,29886.3108,267.2525,-26.6752,NaN
2,Crab,10702305002,A,Crab_sl_PA330,23410.3679,81.8356,21.1272,NaN
3,GX 17+2,90701306002,A,Swift_182251m135218,45107.2367,275.7079,-13.8830,NaN
4,1A 0535+262,90601036001,A,1A_0535p262_SLPA140,2036.1913,86.2426,27.5165,NaN


In [86]:
cutoff_t = Time('2021-01-19T00:00:00.123456789', format='isot', scale='utc')
cutoff_mjd = cutoff_t.mjd

numaster = Table.read('fits/numaster.fits', format = 'fits')
    
dn = numaster.to_pandas()

greenlist = ['NAME', 'RA', 'DEC', 'TIME', 'END_TIME',
       'OBSID', 'EXPOSURE_A', 'EXPOSURE_B', 'PUBLIC_DATE']
for col in dn.columns:
    if col not in greenlist:
        dn.drop(col, axis=1, inplace=True)

dn['OBSID'] = pd.to_numeric(dn['OBSID'])
dn['EXPOSURE_A'] = pd.to_numeric(dn['EXPOSURE_A'])
dn['EXPOSURE_B'] = pd.to_numeric(dn['EXPOSURE_B'])
dn['PUBLIC'] = np.where((dn['PUBLIC_DATE']<cutoff_mjd)&(dn['PUBLIC_DATE'] > 0), 'yes', 'no')
dn.head()



,NAME,RA,DEC,TIME,END_TIME,OBSID,EXPOSURE_A,EXPOSURE_B,PUBLIC_DATE,PUBLIC
0,b'3C273_pcs11 ...,187.2386,2.0479,56109.875778,56109.941750,15011001,2881.8585,2882.4366,56533,yes
1,b'3C273_pcs12 ...,187.2038,2.0472,56109.941750,56110.011194,15012001,2880.8686,2894.2769,56533,yes
2,b'3C273_pcs13 ...,187.2367,2.0157,56110.011194,56110.080639,15013001,2571.8304,2575.1355,56533,yes
3,b'3C273_pcs14 ...,187.3142,2.0520,56110.080639,56110.146611,15014001,2606.3834,2608.6107,56533,yes
4,b'3C273_pcs15 ...,187.2341,2.1272,56110.146611,56110.212583,15015001,2843.7523,2847.6402,56533,yes


In [87]:

df['TIME'] = [-1. for x in range(len(df['SEQID']))]
df['END_TIME'] = [-1. for x in range(len(df['SEQID']))]

for si in np.unique(df['SEQID']):
#    print(si)
#    print(dn.loc[dn['OBSID'] == si, 'TIME'].values[0])

    for mod in ['A', 'B']:
        found = df.loc[(df['SEQID'] == si)&(df['Module'] == mod)]
        if len(found) == 0:
            continue
        
        df.loc[(df['SEQID'] == si)&(df['Module'] == mod), 'TIME'] = dn.loc[dn['OBSID'] == si, 'TIME'].values[0]
        df.loc[(df['SEQID'] == si)&(df['Module'] == mod), 'END_TIME'] = dn.loc[dn['OBSID'] == si, 'END_TIME'].values[0]
        df.loc[(df['SEQID'] == si)&(df['Module'] == mod), 'Exposure'] = dn.loc[dn['OBSID'] == si, f'EXPOSURE_{mod}'].values[0]
        
#        print(df.loc[(df['SEQID'] == si)&(df['Module'] == mod), 'TIME'])
df.head()

,SL Target,SEQID,Module,Primary Target,Exposure,RA,DEC,Notes,TIME,END_TIME
0,GX 5-1,90701331002,A,GRO_J1750m27,29886.3108,267.2525,-26.6752,NaN,59484.209134,59484.917467
1,GR,90701331002,A,GRO_J1750m27,29886.3108,267.2525,-26.6752,NaN,59484.209134,59484.917467
2,Crab,10702305002,A,Crab_sl_PA330,23410.3679,81.8356,21.1272,NaN,59269.278579,59269.806356
3,GX 17+2,90701306002,A,Swift_182251m135218,45107.2367,275.7079,-13.8830,NaN,59265.966079,59267.011217
4,1A 0535+262,90601036001,A,1A_0535p262_SLPA140,2036.1913,86.2426,27.5165,NaN,59186.948717,59187.021634


In [88]:
obj_type = pd.read_csv('csv/target_info.csv')
green_list = ['SL Target', 'Type']
for col in obj_type.columns:
    if col not in green_list:
        if col in obj_type.columns:
            obj_type = obj_type.drop(axis=1,labels=col)
obj_type.head()



,SL Target,Type
0,1RXSJ180408.9-342058,LMXB-NS
1,1A 0535+262,HMXB-NS
2,2MASX J09172716-6456271,AGN
3,2S 0918-549,LMXB-NS
4,3C 273,AGN


In [89]:
# Check for target names that are missing:

for target in df['SL Target']:
    found = obj_type.loc[(target == obj_type['SL Target'])]
    if len(found) == 0:
        print(target)

GR
GR
Unkn
Unkn
Unkn
Complex
Complex
Unkn
Complex
Unkn
Complex
Complex
Complex
Complex
Complex
Complex
Unkn
Unkn
Unkn
Complex
Complex
Complex
Complex
Complex
Complex
Complex
Complex
Complex
Complex
Unkn
Faint
GR
GR
Unkn
Unkn


In [91]:
# Add target classification and target type

df2 = df.copy()
nrows = len(df2['SEQID'])
print(nrows)

classification= ['SL' for x in range(nrows) ]
targ_type= ['??' for x in range(nrows) ]


df2.insert(loc=0, column='Classification', value=classification)
df2.insert(loc=2, column='Target Type', value=targ_type)




for target, ttype in zip(obj_type['SL Target'], obj_type['Type']):
    df2.loc[df2['SL Target'] == target, 'Target Type'] = ttype

    
for id in ['Unkn', 'Complex', 'Faint', 'GR']:
    df2.loc[df2['SL Target'] == id, 'Classification'] = id
    df2.loc[df2['SL Target'] == id, 'SL Target'] = '??'



df2

106


,Classification,SL Target,Target Type,SEQID,Module,Primary Target,Exposure,RA,DEC,Notes,TIME,END_TIME
0,SL,GX 5-1,LMXB-NS,90701331002,A,GRO_J1750m27,29886.3108,267.2525,-26.6752,NaN,59484.209134,59484.917467
1,GR,??,??,90701331002,A,GRO_J1750m27,29886.3108,267.2525,-26.6752,NaN,59484.209134,59484.917467
2,SL,Crab,PWNe,10702305002,A,Crab_sl_PA330,23410.3679,81.8356,21.1272,NaN,59269.278579,59269.806356
3,SL,GX 17+2,LMXB-NS,90701306002,A,Swift_182251m135218,45107.2367,275.7079,-13.8830,NaN,59265.966079,59267.011217
4,SL,1A 0535+262,HMXB-NS,90601036001,A,1A_0535p262_SLPA140,2036.1913,86.2426,27.5165,NaN,59186.948717,59187.021634
...,...,...,...,...,...,...,...,...,...,...,...,...
101,SL,4U 1746-370,LMXB-NS,90601333002,B,IGR_J17379m3747,19726.9895,264.4570,-37.7981,NaN,59140.493856,59140.927884
102,SL,PSR 1509-58,PWNe,30601016002,B,IGR_J15038m6021,42871.4032,226.0118,-60.3875,NaN,59060.434829,59061.431356
103,SL,NGC 1275,AGN,80701509002,B,AT2021ehb,42755.4491,46.8522,40.3047,NaN,59536.827190,59537.875801
104,SL,2S 0918-549,LMXB-NS,90701315002,A,MAXI_J0903m531,23332.0060,136.1776,-53.5405,NaN,59337.122329,59337.587606


In [92]:
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
import astropy.units as u


df2['SIMBAD_ID'] = 'NA'
df2['RA_SL'] = -999
df2['DEC_SL'] = -999

targets = df2['SL Target'].unique()


for target in targets:
    if 'Crab' in target:
        starget = 'Crab'
    elif '1145.1' in target:
        starget = '1E 1145.1-6141'
    elif target == '??':
        continue
    elif '1716' in target:
        starget = 'GRS 1716-249'
    elif '16320' in target:
        starget = 'IGR J16320-4751'
    elif 'M1812' in target:
        starget = '4U 1812-12'
    elif '1348' in target:
        starget = 'MAXI J1348-630'
    elif 'J1535' in target:
        starget = 'MAXI J1535-571'
    elif 'J1820' in target:
        starget = 'MAXI J1820+070'
    elif 'Coma' in target:
        starget = 'Coma Cluster'
    elif 'MAXI J1621' in target:
        starget = 'MAXI J1621-501'
    elif 'X1908' in target:
        starget = '1E 1908.4+0730'
    else:
        starget = target
        
    result_table = Simbad.query_object(starget)


    ra_tab = result_table['RA'].data.data[0]
    dec_tab = result_table['DEC'].data.data[0]
    coord = SkyCoord(ra_tab, dec_tab, unit=(u.hourangle, u.deg))
    
    
    df2.loc[df2['SL Target'] == target, 'SIMBAD_ID'] = starget
    df2.loc[df2['SL Target'] == target, 'RA_SL'] = coord.ra.deg
    df2.loc[df2['SL Target'] == target, 'DEC_SL'] = coord.dec.deg



#    print(f'{target} {starget} {coord.ra.deg} {coord.dec.deg}')
    #     ra = np.append(ra, coord.ra.deg)
#     dec = np.append(dec, coord.dec.deg)
#     gal_lat = np.append(gal_lat, coord.galactic.b.deg)
#     gal_lon = np.append(gal_lon, coord.galactic.l.deg)
#     names = np.append(names, target)

In [93]:
df2.sort_values(by=['RA', 'TIME', 'Module'], axis=0, inplace=True, ignore_index=True)

df2.head()

,Classification,SL Target,Target Type,SEQID,Module,Primary Target,Exposure,RA,DEC,Notes,TIME,END_TIME,SIMBAD_ID,RA_SL,DEC_SL
0,SL,NGC 1275,AGN,80701509002,B,AT2021ehb,42755.4491,46.8522,40.3047,NaN,59536.827190,59537.875801,NGC 1275,49.950665,41.511698
1,SL,LMC X-4,NS,90701342002,A,eRASSUJ050810d4m660653,55783.4115,77.1065,-66.1471,NaN,59568.966079,59570.417467,LMC X-4,83.206482,-66.370334
2,SL,Crab,PWNe,10702305002,A,Crab_sl_PA330,23410.3679,81.8356,21.1272,NaN,59269.278579,59269.806356,Crab,83.633083,22.014500
3,SL,Crab,PWNe,10702305002,B,Crab_sl_PA330,23186.3627,81.8356,21.1272,NaN,59269.278579,59269.806356,Crab,83.633083,22.014500
4,SL,Crab,PWNe,10602005002,A,Crab_sl_PA150,14248.1523,85.4469,22.9130,NaN,59088.993856,59089.382745,Crab,83.633083,22.014500


In [94]:
slid = [f'StrayCatsII_{x}' for x in df2.index]
df2.insert(loc = 0, 
          column = 'STRAYID', 
          value = slid)

In [95]:
df2.head()

,STRAYID,Classification,SL Target,Target Type,SEQID,Module,Primary Target,Exposure,RA,DEC,Notes,TIME,END_TIME,SIMBAD_ID,RA_SL,DEC_SL
0,StrayCatsII_0,SL,NGC 1275,AGN,80701509002,B,AT2021ehb,42755.4491,46.8522,40.3047,NaN,59536.827190,59537.875801,NGC 1275,49.950665,41.511698
1,StrayCatsII_1,SL,LMC X-4,NS,90701342002,A,eRASSUJ050810d4m660653,55783.4115,77.1065,-66.1471,NaN,59568.966079,59570.417467,LMC X-4,83.206482,-66.370334
2,StrayCatsII_2,SL,Crab,PWNe,10702305002,A,Crab_sl_PA330,23410.3679,81.8356,21.1272,NaN,59269.278579,59269.806356,Crab,83.633083,22.014500
3,StrayCatsII_3,SL,Crab,PWNe,10702305002,B,Crab_sl_PA330,23186.3627,81.8356,21.1272,NaN,59269.278579,59269.806356,Crab,83.633083,22.014500
4,StrayCatsII_4,SL,Crab,PWNe,10602005002,A,Crab_sl_PA150,14248.1523,85.4469,22.9130,NaN,59088.993856,59089.382745,Crab,83.633083,22.014500


In [96]:
df2 = df2.rename(columns={"SL Target": "SL Source"})
df2 = df2.rename(columns={"Primary Target": "Primary"})
df2 = df2.rename(columns={"RA": "RA_PRIMARY"})
df2 = df2.rename(columns={"DEC": "DEC_PRIMARY"})
df2 = df2.rename(columns={"Target Type": "SL Type"})
df2

,STRAYID,Classification,SL Source,SL Type,SEQID,Module,Primary,Exposure,RA_PRIMARY,DEC_PRIMARY,Notes,TIME,END_TIME,SIMBAD_ID,RA_SL,DEC_SL
0,StrayCatsII_0,SL,NGC 1275,AGN,80701509002,B,AT2021ehb,42755.4491,46.8522,40.3047,NaN,59536.827190,59537.875801,NGC 1275,49.950665,41.511698
1,StrayCatsII_1,SL,LMC X-4,NS,90701342002,A,eRASSUJ050810d4m660653,55783.4115,77.1065,-66.1471,NaN,59568.966079,59570.417467,LMC X-4,83.206482,-66.370334
2,StrayCatsII_2,SL,Crab,PWNe,10702305002,A,Crab_sl_PA330,23410.3679,81.8356,21.1272,NaN,59269.278579,59269.806356,Crab,83.633083,22.014500
3,StrayCatsII_3,SL,Crab,PWNe,10702305002,B,Crab_sl_PA330,23186.3627,81.8356,21.1272,NaN,59269.278579,59269.806356,Crab,83.633083,22.014500
4,StrayCatsII_4,SL,Crab,PWNe,10602005002,A,Crab_sl_PA150,14248.1523,85.4469,22.9130,NaN,59088.993856,59089.382745,Crab,83.633083,22.014500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,StrayCatsII_101,SL,GX 17+2,LMXB-NS,90701306002,B,Swift_182251m135218,44607.4871,275.7079,-13.8830,NaN,59265.966079,59267.011217,GX 17+2,274.005787,-14.036283
102,StrayCatsII_102,GR,??,??,40701002002,A,W50_WESTERN_LOBE,97596.2824,287.6642,5.0250,NaN,59409.806356,59412.212606,NA,-999.000000,-999.000000
103,StrayCatsII_103,GR,??,??,40701002002,B,W50_WESTERN_LOBE,96644.4133,287.6642,5.0250,NaN,59409.806356,59412.212606,NA,-999.000000,-999.000000
104,StrayCatsII_104,SL,Cyg X-3,HMXB-BH,90701336002,B,EXO_2030p375,23583.9090,308.0343,37.5922,NaN,59526.146634,59526.747329,Cyg X-3,308.107417,40.957750


In [97]:
exp = df2['Exposure']
df2.drop(labels=['Exposure'], axis=1,inplace = True)
df2.insert(6, 'Exposure', exp)

sl = df2['SL Source']
df2.drop(labels=['SL Source'], axis=1,inplace = True)
df2.insert(12, 'SL Source', sl)


sltype = df2['SL Type']
df2.drop(labels=['SL Type'], axis=1,inplace = True)
df2.insert(12, 'SL Type', sltype)


df2.head()

,STRAYID,Classification,SEQID,Module,Exposure,Primary,RA_PRIMARY,DEC_PRIMARY,Notes,TIME,END_TIME,SL Source,SL Type,SIMBAD_ID,RA_SL,DEC_SL
0,StrayCatsII_0,SL,80701509002,B,42755.4491,AT2021ehb,46.8522,40.3047,NaN,59536.827190,59537.875801,NGC 1275,AGN,NGC 1275,49.950665,41.511698
1,StrayCatsII_1,SL,90701342002,A,55783.4115,eRASSUJ050810d4m660653,77.1065,-66.1471,NaN,59568.966079,59570.417467,LMC X-4,NS,LMC X-4,83.206482,-66.370334
2,StrayCatsII_2,SL,10702305002,A,23410.3679,Crab_sl_PA330,81.8356,21.1272,NaN,59269.278579,59269.806356,Crab,PWNe,Crab,83.633083,22.014500
3,StrayCatsII_3,SL,10702305002,B,23186.3627,Crab_sl_PA330,81.8356,21.1272,NaN,59269.278579,59269.806356,Crab,PWNe,Crab,83.633083,22.014500
4,StrayCatsII_4,SL,10602005002,A,14248.1523,Crab_sl_PA150,85.4469,22.9130,NaN,59088.993856,59089.382745,Crab,PWNe,Crab,83.633083,22.014500


In [107]:

v2 = Table.read('fits/straycats_v2.fits', format = 'fits')
